In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install warnings
!pip install dgl
!pip install texttable

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

2.0.1+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.0/627.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.1/504.1 kB 897.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 21.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings
Looking in indexes: https://pypi.org/simple,

In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [2]:
import random
from torchvision import transforms, datasets

import os
import pickle
from scipy.spatial.distance import cdist
from scipy import ndimage
import numpy as np
from torch_geometric.utils import subgraph,k_hop_subgraph,sort_edge_index
import dgl
import torch
import time
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [3]:
import numpy as np
import os.path as osp
import pickle
import torch
import torch.utils
import torch.utils.data
import torch.nn.functional as F
from scipy.spatial.distance import cdist
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import InMemoryDataset,Data
#/content/drive/MyDrive/Colab_Notebooks/mnist08_83_75sp_train.pkl


In [4]:
import torch
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
from torch_geometric.nn.inits import glorot, zeros
import pdb

def mean(l):
  return sum(l)/len(l)
class GCNConv(MessagePassing):

    def __init__(self,
                 in_channels,
                 out_channels,
                 improved=False,
                 cached=False,
                 bias=True,
                 edge_norm=True,
                 gfn=False):
        super(GCNConv, self).__init__('add')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.edge_norm = edge_norm
        self.gfn = gfn
        self.message_mask = None
        self.weight = Parameter(torch.Tensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)
        self.cached_result = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, improved=False, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ),
                                     dtype=dtype,
                                     device=edge_index.device)

        edge_weight = edge_weight.view(-1)


        assert edge_weight.size(0) == edge_index.size(1)

        edge_index, edge_weight = remove_self_loops(edge_index, edge_weight)
        edge_index, _ = add_self_loops(edge_index, num_nodes=num_nodes)
        # Add edge_weight for loop edges.
        loop_weight = torch.full((num_nodes, ),
                                 1 if not improved else 2,
                                 dtype=edge_weight.dtype,
                                 device=edge_weight.device)
        edge_weight = torch.cat([edge_weight, loop_weight], dim=0)

        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight=None):
        """"""

        x = torch.matmul(x, self.weight)
        if self.gfn:
            return x

        if not self.cached or self.cached_result is None:
            if self.edge_norm:
                edge_index, norm = GCNConv.norm(
                    edge_index,
                    x.size(0),
                    edge_weight,
                    self.improved,
                    x.dtype)
            else:
                norm = None
            self.cached_result = edge_index, norm

        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):

        if self.edge_norm:
            return norm.view(-1, 1) * x_j
        else:
            return x_j

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
from torch_geometric.data import Data,Batch,DataLoader
import torch
import math
import math
import numpy as np
from torch_geometric.utils import (remove_self_loops, degree,
                                   batched_negative_sampling)
from torch_geometric.utils.num_nodes import maybe_num_nodes
def split_batch(edge_index,batch):
  split = degree(batch[edge_index[0]], dtype=torch.long).tolist()
  edge_indices = torch.split(edge_index, split, dim=1)
  num_nodes = degree(batch, dtype=torch.long)
  cum_nodes = torch.cat([batch.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]])
  num_edges = torch.tensor([e.size(1) for e in edge_indices], dtype=torch.long).to(device)
  cum_edges = torch.cat([batch.new_zeros(1), num_edges.cumsum(dim=0)[:-1]])
  return edge_indices, num_nodes, cum_nodes, num_edges, cum_edges
def split_graph_bybatch(edge_weight_o,xo_node_att,xc_node_att,edge_index,batch,ratio=0.5):

  causal_edge_index = torch.LongTensor([[],[]]).to(device)
  causal_edge_weight = torch.tensor([]).to(device)
  conf_edge_index = torch.LongTensor([[],[]]).to(device)
  conf_edge_weight = torch.tensor([]).to(device)

  #causal_edge_attr = torch.tensor([])
  topnode=torch.LongTensor([]).to(device)
  notopnode=torch.LongTensor([]).to(device)

  edge_indices, _, _, num_edges, cum_edges = split_batch(edge_index,batch)
  for edge_index, N, C in zip(edge_indices, num_edges, cum_edges):
      n_reserve =  int(ratio * N)
      edge_attr = edge_weight_o[C:C+N]
      single_mask = edge_weight_o[C:C+N]
      single_mask_detach = edge_weight_o[C:C+N].detach().cpu().numpy()
      rank = np.argpartition(-single_mask_detach, n_reserve)
      idx_reserve, idx_drop = rank[:n_reserve], rank[n_reserve:]

      causal_edge_index = torch.cat([causal_edge_index, edge_index[:, idx_reserve]], dim=1)
      conf_edge_index = torch.cat([conf_edge_index, edge_index[:, idx_drop]], dim=1)
      #print(causal_edge_index)
      causal_edge_weight = torch.cat([causal_edge_weight, single_mask[idx_reserve]])
      conf_edge_weight = torch.cat([conf_edge_weight, 1-single_mask[idx_drop]])
      causal_sub_nodes = torch.unique(edge_index[:, idx_reserve])
      non_sub_nodes=torch.unique(edge_index[:, idx_drop])
      #print(xo_node_att[causal_sub_nodes])
      #print(f'non_subnode:{non_sub_nodes}')
    # print(f'non weight:{ -1 * single_mask[idx_drop]}')
      #print(f'subnode:{causal_sub_nodes}')
      #print(f'c subnode weight:{single_mask[idx_reserve]}')
      topnon=torch.topk(xc_node_att[non_sub_nodes],int(len(non_sub_nodes)*0.5))[1]

      top=torch.topk(xo_node_att[causal_sub_nodes],int(len(causal_sub_nodes)*0.5))[1]

      topnode = torch.cat([topnode, causal_sub_nodes[top]])
      notopnode=torch.cat([notopnode, non_sub_nodes[topnon]])
      #print(topnode)
      #print(f'top node:{topnode}')
      #print(f'nontop node:{notopnode}')
  return (causal_edge_index,causal_edge_weight,topnode),(conf_edge_index,conf_edge_weight,notopnode)
def k_hop_subgraph(node_idx,num_hops,edge_index,ew,relabel_nodes=False,num_nodes = None):

    num_nodes = maybe_num_nodes(edge_index, num_nodes)


    col, row = edge_index

    node_mask = row.new_empty(num_nodes, dtype=torch.bool)
    edge_mask = row.new_empty(row.size(0), dtype=torch.bool)

    if isinstance(node_idx, (int, list, tuple)):
        node_idx = torch.tensor([node_idx], device=row.device).flatten()
    else:
        node_idx = node_idx.to(row.device)

    subsets = [node_idx]

    for _ in range(num_hops):
        node_mask.fill_(False)
        node_mask[subsets[-1]] = True
        torch.index_select(node_mask, 0, row, out=edge_mask)
        subsets.append(col[edge_mask])

    subset, inv = torch.cat(subsets).unique(return_inverse=True)
    inv = inv[:node_idx.numel()]

    node_mask.fill_(False)
    node_mask[subset] = True


    edge_mask = node_mask[row] & node_mask[col]

    edge_index = edge_index[:, edge_mask]

    edge_weight=torch.masked_select(ew,edge_mask)
    if relabel_nodes:
        node_idx = row.new_full((num_nodes, ), -1)
        node_idx[subset] = torch.arange(subset.size(0), device=row.device)
        edge_index = node_idx[edge_index]

    return subset, edge_index, inv,edge_weight
def combine_subg(databatch,node_att,select_node,edge_indexn,eweight,tragety,uor):
  undata=[]
  select_node_batch_number=databatch[select_node]
  edidx=edge_indexn.clone()
  ew=eweight.clone()
  for i in range(len(select_node)):
    #print(i)
    #subset,edge_index,_,edge_weight=k_hop_subgraph(int(select_node[i]),5,edge_index,eweight,relabel_nodes=False)
    subset,edge_index,_,edge_weight=k_hop_subgraph(int(select_node[i]),3,edidx,ew,relabel_nodes=True)
    undata.append(Data(x=node_att[subset],edge_index=edge_index,edge_attr=edge_weight,y=tragety[int(select_node_batch_number[i])]))
  #print(int(len(select_node)/2))
  if(uor=='c'):
    d=DataLoader(undata,batch_size=2,shuffle=True,drop_last=True)
  else:
    if(len(undata)>10):
      bs=int(len(undata)*0.1)
    else:
      bs=1
    d=DataLoader(undata,batch_size=bs,shuffle=True,drop_last=True)
  return d

  #fin=Batch.from_data_list(undata)
  return d
def uncert(a):
  va=torch.var(a)
  ma=torch.mean(a)
  l=[]
  for i in a:
    l.append((i-ma)/va)
  l=torch.tensor(l)
  #print(l)
  l=torch.abs(l)
  res=torch.topk(l,int(len(a)*0.6),largest=False)
  #print(res)
  return a[res[1]]

In [6]:
from functools import partial
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d, Sequential, ReLU
from torch_geometric.nn import global_mean_pool, global_add_pool, GINConv, GATConv

import random
import pdb
#device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
from torch_geometric.utils import to_dgl,from_dgl


layers=3
with_random=True
fc_num=222
hidden=32
eval_random=False
'''def subg(data,weight):
  re_data=data.clone()
  edge_score = weight.clone()
  re_data=to_dgl(re_data)
  mean_score = torch.mean(edge_score)
  sorted_score, index = torch.sort(edge_score.view(-1))
  prune_num_edges = int(torch.sum(torch.lt(edge_score, mean_score)))
  prune_index=index[:prune_num_edges]
  re_data.remove_edges(prune_index)
  isolated_nodes = ((re_data.in_degrees() == 0) & (re_data.out_degrees() == 0)).nonzero().squeeze(1)
  re_data=from_dgl(re_data)
  return re_data

def th_delete(tensor, number):
  #print(tensor.numel())
  mask = torch.isin(tensor,number,invert=True)
  return tensor[mask]
def mask_graph(graph_x,select_node):
  result=graph_x.clone()
  result[select_node]=0
  return result#.to(device)v

def split_graph(graph_x,node_of_graph,edge_index,edge_weight,type_of_graph=True):
  select_node_number=0
  if(type_of_graph==True):
    select_node_number=int(node_of_graph/10)#3
    if(select_node_number==0):
      select_node_number=int(node_of_graph/3)
    select_node_numberms=node_of_graph-select_node_number

    select_nodems=torch.topk(torch.mean(graph_x,dim=1),select_node_numberms,largest=False)[1]
    select_node=torch.topk(torch.mean(graph_x,dim=1),select_node_number,largest=True)[1]
    sedge_index,sedge_weight=subgraph(subset=select_node,edge_index=edge_index,edge_attr=edge_weight)

    #print(select_node)
    #return graph_x,select_node,sedge_index,sedge_weight
    return mask_graph(graph_x,select_nodems),select_node,sedge_index,sedge_weight
  else:
    select_node_number=int(node_of_graph/10)#2
    if(select_node_number==0):
      select_node_number=int(node_of_graph/3)
    select_node_numberms=node_of_graph-select_node_number

    select_nodems=torch.topk(torch.mean(graph_x,dim=1),select_node_numberms,largest=False)[1]
    select_node=torch.topk(torch.mean(graph_x,dim=1),select_node_number,largest=True)[1]
    sedge_index,sedge_weight=subgraph(subset=select_node,edge_index=edge_index,edge_attr=edge_weight)
    return mask_graph(graph_x,select_nodems),select_node,sedge_index,sedge_weight
    #return graph_x,select_node,sedge_index,sedge_weight

def uncertainty_mask_gnerate(graph_x,node_in_graph,edge_index,edge_weight):
  all_mask=[]
  edge_indexso,edge_weightso=sort_edge_index(edge_index,edge_weight)
  #max_nodeidx=int(max(node_in_graph))+1
  edge_node=torch.unique(edge_index)
  for i in node_in_graph:
    if(i in edge_node):
      subset,edge_indexk,inv,mask=k_hop_subgraph(node_idx=int(i),num_hops=5,edge_index=edge_indexso)#,num_nodes=max_nodeidx)
    #random_mask=random.sample(node_in_graph.tolist(),int(len(node_in_graph)/3))
      #sedge_index,sedge_weight=subgraph(subset=subset,edge_index=edge_index,edge_attr=edge_weight)#,num_nodes=max_nodeidx)
      sedge_weight=torch.masked_select(edge_weightso,mask==True)
      #all_mask.append((int(i),graph_x,edge_indexk,sedge_weight))
      all_mask.append((int(i),mask_graph(graph_x,th_delete(node_in_graph,subset)),edge_indexk,sedge_weight))
    else:
      #all_mask.append((int(i),graph_x,edge_index,edge_weight))
      all_mask.append((int(i),graph_x,edge_index,edge_weight))
        return all_mask'''


class causalpreCO(torch.nn.Module):
  def __init__(self, num_features,num_classes,gfn=False,collapse=False,residual=False,
                      res_branch="BNConvReLU",
                      global_pool="sum",
                      dropout=0,
                      edge_norm=True):
    super(causalpreCO, self).__init__()
    num_conv_layers = layers
    self.global_pool = global_add_pool
    hidden=32

    self.with_random = with_random
    GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)
    self.bnc = BatchNorm1d(hidden)
    self.bno= BatchNorm1d(hidden)
    self.context_convs = GConv(hidden, hidden)
    self.objects_convs = GConv(hidden, hidden)


    hidden_in = num_features
    self.num_classes = num_classes
    hidden_out = num_classes
    self.fc_num = fc_num

    self.object_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )
    self.context_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )
    self.random_readout_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )



    # BN initialization.
    for m in self.modules():
        if isinstance(m, (torch.nn.BatchNorm1d)):
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias, 0.0001)

  def forward(self,causal,noncausal,edge_index,causal_edge_weight,noncausal_edge_weight,batch,eval_random=True):


    #xo1 = F.relu(self.objects_convs(causal,edge_index, causal_edge_weight))
    #xo2 = self.global_pool(xo1,batch)
    xc_logis = self.context_readout_layer(noncausal,edge_index,noncausal_edge_weight,batch)
    xo_logis = self.objects_readout_layer(causal,edge_index,causal_edge_weight,batch)
    xco_logis = self.random_readout_layer(causal,noncausal,edge_index,causal_edge_weight,noncausal_edge_weight,batch,eval_random)

    #return xc_logis, xo_logis, xco_logi
    return xo_logis,xc_logis,xco_logis#self.objects_readout_layer(causal,edge_index,causal_edge_weight,batch)


  def objects_readout_layer(self,causal,edge_index,causal_edge_weight,batch):
      xo = F.relu(self.objects_convs(self.bno(causal),edge_index,causal_edge_weight))
      xo = self.global_pool(xo,batch)
      x_logis = F.log_softmax(self.object_mlp(xo), dim=-1)
      return x_logis
  def context_readout_layer(self,noncausal,edge_index,noncausal_edge_weight,batch):
      xc = F.relu(self.context_convs(self.bnc(noncausal), edge_index,noncausal_edge_weight))
      xc = self.global_pool(xc,batch)

      x_logis = F.log_softmax(self.context_mlp(xc), dim=-1)
      return x_logis
  def random_readout_layer(self,causal,noncausal,edge_index,causal_edge_weight,noncausal_edge_weight,batch,eval_random):

      xc = F.relu(self.context_convs(self.bnc(noncausal), edge_index, noncausal_edge_weight))
      xo = F.relu(self.objects_convs(self.bno(causal), edge_index, causal_edge_weight))

      xc = self.global_pool(xc, batch)
      xo = self.global_pool(xo, batch)

      num = xc.shape[0]
      l = [i for i in range(num)]
      if self.with_random:
          if eval_random:
              random.shuffle(l)
      random_idx = torch.tensor(l)
      #if self.args.cat_or_add == "cat":
      #    x = torch.cat((xc[random_idx], xo), dim=1)
      #else:
      x = xc[random_idx] + xo

      x_logis = F.log_softmax(self.random_readout_mlp(x), dim=-1)
      return x_logis



  def uncertainty_L_hop(self,target,causal,node,edge_indexn,causal_edge_weightn,batch,c_or_uc='c'):
    fin=combine_subg(batch,causal,node,edge_indexn,causal_edge_weightn,target,uor=c_or_uc)
    loss_list=[]

    for i in fin:
      lo=self.objects_readout_layer(causal=i.x,edge_index=i.edge_index,causal_edge_weight=i.edge_attr,batch=i.batch)
      t=i.y.view(-1)
      loss=F.nll_loss(lo,t)
      loss_list.append(loss)

    loss_list=torch.tensor(loss_list)
    #v=torch.mean(loss_list)
    #return v

    if(c_or_uc=='c'):
      v=uncert(loss_list)
      return torch.mean(v)
    else:
      v=torch.topk(loss_list,int(len(loss_list)*0.2))[0]
      return torch.mean(v)

  def non_uncertainty_L_hop(self,target,noncausal,node,edge_indexn,noncausal_edge_weightn,batch,c_or_uc='c'):
    fin=combine_subg(batch,noncausal,node,edge_indexn,noncausal_edge_weightn,target,uor='c_or_uc')
    loss_list=[]
    for i in fin:
      lo=self.context_readout_layer(noncausal=i.x,edge_index=i.edge_index,noncausal_edge_weight=i.edge_attr,batch=i.batch)
      uniform_target = torch.ones_like(lo, dtype=torch.float).to(device) /2

      loss=F.kl_div(lo, uniform_target, reduction='batchmean')
      #print(loss)
      loss_list.append(loss)

    loss_list=torch.tensor(loss_list)
    v=torch.mean(loss_list)
    #return v
    #print(loss_list)
    if(c_or_uc=='c'):
      v=uncert(loss_list)
      return torch.mean(v)
    else:
      v=torch.topk(loss_list,int(len(loss_list)*0.2))[0]
      return torch.mean(v)





In [7]:

class CausalGAN(torch.nn.Module):
    """GCN with BN and residual connection."""
    def __init__(self, num_features,
                       num_classes,
                       gfn=False,
                       collapse=False,
                       residual=False,
                       res_branch="BNConvReLU",
                       global_pool="sum",
                       dropout=0.4,
                       edge_norm=True):
        super(CausalGAN, self).__init__()
        num_conv_layers = layers
        #hidden = args.hidden
        #self.args = args
        hidden=32
        head=4
        self.global_pool = global_add_pool
        self.dropout = dropout
        self.with_random = with_random
        self.without_node_attention = False
        self.without_edge_attention = False
        GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)

        hidden_in = num_features
        self.num_classes = num_classes
        hidden_out = num_classes
        self.fc_num = fc_num
        self.bn_feat = BatchNorm1d(hidden_in)
        self.conv_feat = GCNConv(hidden_in, hidden, gfn=True) # linear transform
        self.bns_conv = torch.nn.ModuleList()
        self.convs = torch.nn.ModuleList()

        for i in range(num_conv_layers):
            self.bns_conv.append(BatchNorm1d(hidden))
            self.convs.append(GATConv(hidden, int(hidden / head), heads=head, dropout=dropout))

        self.edge_att_mlp = nn.Linear(hidden * 2, 2)
        self.node_att_mlp = nn.Linear(hidden, 2)
        self.bnc = BatchNorm1d(hidden)
        self.bno= BatchNorm1d(hidden)
        self.context_convs = GConv(hidden, hidden)
        self.objects_convs = GConv(hidden, hidden)
        self.relu = nn.ReLU(inplace=False)

        for m in self.modules():
            if isinstance(m, (torch.nn.BatchNorm1d)):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0.0001)

    def forward(self, data, eval_random=True):

        x = data.x if data.x is not None else data.feat
        edge_index,edge_attr,batch = data.edge_index,data.edge_attr,data.batch
        row, col = edge_index
        x = self.bn_feat(x)
        x = self.relu(self.conv_feat(x, edge_index,edge_weight=edge_attr))

        for i, conv in enumerate(self.convs):
            x = self.bns_conv[i](x)
            x = self.relu(conv(x, edge_index))

        edge_rep = torch.cat([x[row], x[col]], dim=-1)

        if self.without_edge_attention:
            edge_att = 0.5 * torch.ones(edge_rep.shape[0], 2).to(device)
        else:
            edge_att = F.softmax(self.edge_att_mlp(edge_rep), dim=-1)
        edge_weight_c = edge_att[:, 0]
        edge_weight_o = edge_att[:, 1]

        if self.without_node_attention:
            node_att = 0.5 * torch.ones(x.shape[0], 2).to(device)
        else:
            node_att = F.softmax(self.node_att_mlp(x), dim=-1)
        xc = node_att[:, 0].view(-1, 1) * x
        xo = node_att[:, 1].view(-1, 1) * x
        xo_node_att=node_att[:, 1]
        xc_node_att=node_att[:, 0]

        #print(edge_weight_o)
        #xc = F.relu(self.context_convs(self.bnc(xc), edge_index, edge_weight_c))
        #xo = F.relu(self.objects_convs(self.bno(xo), edge_index, edge_weight_o))
        #node_of_graph=xo.shape[0]
        #causal_part,causal_node=split_graph(graph_x=xo,node_of_graph=node_of_graph)
        #noncausal_part,noncausal_node=split_graph(graph_x=xc,node_of_graph=node_of_graph,type_of_graph=False)

        #xc = self.global_pool(xc, batch)
        #xo = self.global_pool(xo, batch)

        #xc_logis = self.context_readout_layer(xc)
        #xo_logis = self.objects_readout_layer(xo)
        #xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

      # return (xo,edge_index,edge_weight_o),(xc,edge_index,edge_weight_c),batch
        return (xo,xo_node_att,edge_weight_o),(xc,xc_node_att,edge_weight_c),edge_index,batch
        #return (causal_attention,causal_part,causal_node),(noncausal_attention,noncausal_part,noncausal_node),batch
        #return (xo,causal_part,causal_node),(xc,noncausal_part,noncausal_node),batch



In [ ]:
from torch_geometric.utils import subgraph

class learning_hetero(torch.nn.Module):
   def __init__(self,number_of_class=1):
     super(learning_hetero, self).__init__()
     self.mlp1 = nn.Linear(hidden*2, number_of_class)
     #self.lkrelu = nn.LeakyReLU()
     #self.mlp2 = nn.Linear(hidden, number_of_class)
     self.sigmod = nn.Sigmoid()

   def forward(self,graph,edge_index,batch):
     #edge_ho=self.get_label(data,batch)
     row, col = edge_index
     data=torch.cat([graph[row], graph[col]], dim=-1)
     x=self.mlp1(data)
     #x=self.sigmod(x)
     return x


   def get_label(self,edge_index,graph,batch):
      edge_number=edge_index.shape[1]
      attr_mean=torch.mean(graph,dim=1)
      homo_threshold=torch.var(attr_mean)*2
      edge_classif=torch.zeros(edge_number)
      dic={}
      for i in range(len(attr_mean)):
        dic[int(i)]=attr_mean[i]

      for i in range(len(edge_classif)):
        if(abs(dic[int(edge_index[0][i])]-dic[int(edge_index[1][i])])>homo_threshold):
          edge_classif[i]=1
      return edge_classif


In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from drive.MyDrive.gsst2.graphsst2_dataset import get_dataset, get_dataloader
path ='/content/drive/MyDrive/gsst2/'
dataset = get_dataset(dataset_dir=path, dataset_name='Graph_SST2', task=None)
dataloader = get_dataloader(dataset,
                            batch_size=128,
                            degree_bias=True)


28327 3147 12305


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
t1=dataset[2]
t1

Data(x=[12, 768], edge_index=[2, 22], y=[1], sentence_tokens=[12], edge_attr=[22, 1])

In [ ]:
t1.y

tensor([1])

In [ ]:
t1.sentence_tokens

['that',
 'loves',
 'its',
 'characters',
 'and',
 'communicates',
 'something',
 'rather',
 'beautiful',
 'about',
 'human',
 'nature']

In [ ]:
torch.topk(t1.x.mean(axis=1),5)

torch.return_types.topk(
values=tensor([-0.0083, -0.0087, -0.0089, -0.0094, -0.0095]),
indices=tensor([1, 3, 5, 2, 8]))

In [9]:
number_of_class=2#768
causal_model = CausalGAN(768,number_of_class).to(device)
predictco_model=causalpreCO(768,number_of_class).to(device)
model_optimizer = torch.optim.Adam(
            list(causal_model.parameters())+list(predictco_model.parameters()),
            lr=0.001)

Epo=500
lr_scheduler = CosineAnnealingLR(model_optimizer, T_max=Epo, eta_min=1e-6, last_epoch=-1, verbose=False)

loss_he_ho = nn.BCEWithLogitsLoss()
m=nn.Sigmoid()
#attention_opt=torch.optim.Adam(list(causal_model.edge_att_mlp.parameters())+list(causal_model.node_att_mlp.parameters()),lr=0.001)
train_loader = dataloader['train']
val_loader = dataloader['eval']
test_loader = dataloader['test']

In [10]:
train_loader = dataloader['train']
val_loader = dataloader['eval']
test_loader = dataloader['test']

In [9]:
number_of_class=2
causal_model = CausalGAN(768,number_of_class).to(device)
predictco_model=causalpreCO(768,number_of_class).to(device)
checkpoint = torch.load('/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose/after_mygsst_new.pt')
causal_model.load_state_dict(checkpoint['causal_model.state_dic'])
predictco_model.load_state_dict(checkpoint['predictco_model_state_dict()'])


<All keys matched successfully>

In [13]:
correct=0
correct_c=0
correct_o=0
correct_o_pred=0
print(f"------tvalation---------{0}")
causal_model.eval()
predictco_model.eval()
for data in test_loader:
  data = data.to(device)
  (xo,xo_node_att,edge_weight_o),(xc,xc_node_att,edge_weight_c),edge_index,batch=causal_model(data)
  #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
  #co_logs = predictredom_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
  #causal attention graph prediction
  o_logs,c_logs,co_logs=predictco_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
  (causal_edge_index,causal_edge_weight,topnode),(conf_edge_index,conf_edge_weight,notopnode)=split_graph_bybatch(edge_weight_o,xo_node_att,xc_node_att,edge_index,batch,ratio=0.5)
  sub,edge_i,_,edge_w=k_hop_subgraph(topnode,2,causal_edge_index,causal_edge_weight,relabel_nodes=True)
  o_logs2=predictco_model.objects_readout_layer(causal=xo[sub],edge_index=edge_i,causal_edge_weight=edge_w,batch=batch[sub])
  #causal,edge_index,causal_edge_weight,batch
  # o_logs2=predictco_model.objects_readout_layer(causal=only_causl,edge_index=causal_edge_index,causal_edge_weight=causal_edge_weight,batch=batch)
  #c_logs=predictno_model(noncausal=xc,edge_index=edge_index,noncausal_edge_weight=edge_weight_c,batch=batch)

  pred = co_logs.max(1)[1]
  pred_c = c_logs.max(1)[1]
  pred_o = o_logs.max(1)[1]
  pre_ooo=o_logs2.max(1)[1]
  correct += pred.eq(data.y.view(-1)).sum().item()
  correct_c += pred_c.eq(data.y.view(-1)).sum().item()
  correct_o += pred_o.eq(data.y.view(-1)).sum().item()
  correct_o_pred+=pre_ooo.eq(data.y.view(-1)).sum().item()
acc_co = correct / len(test_loader.dataset)
acc_c = correct_c / len(test_loader.dataset)
acc_o = correct_o / len(test_loader.dataset)
acc_only=correct_o_pred/len(test_loader.dataset)
print(f"causal val accuracy:{acc_co}")
print(f"causal val only causal accuracy:{acc_o}")
print(f"causal val only causal accuracy:{acc_only}")

------tvalation---------0
causal val accuracy:0.8055262088581877
causal val only causal accuracy:0.8008939455505892
causal val only causal accuracy:0.8014628199918732


In [10]:
#use the node attention
loss_value=[]
orio=[]
orin=[]
ocrin=[]
only_c=[]
only_n=[]
cer=[]
ucer=[]
hetr=[]

loss_value_valation=[]

loss_value2=[]
loss_value_valation2=[]
c=0.5
o=1
co=0.5
import time
import json
import gc
from tqdm import tqdm
Epo=100
#scaler = torch.cuda.amp.GradScaler()

for epoch in range(Epo):
  #model.train()
  total_loss = 0
  total_loss_c = 0
  total_loss_o = 0
  total_loss_co = 0
  total_loss_c2 = 0 #c_loss2.item() #* num_graphs(data)
  total_loss_o2 = 0 #o_loss2.item() #* num_graphs(data)
  o_confin =0 #final_conf_uncer.item()# * num_graphs(data)
  c_unfin =0 #final_conf_uncer_un.item()
  heter_all=0 #heter_loss.item()
  correct_o = 0
  nb=0
  start = time.time()
  loop = tqdm(enumerate(train_loader), total=len(train_loader))
  causal_model.train()
  predictco_model.train()
  print(f"-----training-------{epoch}")
  for it, data in loop:

    nb+=1
    model_optimizer.zero_grad()
    data = data.to(device)

    one_hot_target = data.y.view(-1)
    #causal and non-causal attention graph generate
    # with torch.autocast(device_type='cuda', dtype=torch.float16):
    (xo,xo_node_att,edge_weight_o),(xc,xc_node_att,edge_weight_c),edge_index,batch=causal_model(data)
    #non-causal attention graph prediction
    #xo_logis,xc_logis,xco_logis
    #causal attention graph prediction


    o_logs,c_logs,co_logs=predictco_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
    #c_logs=predictno_model(noncausal=xc,edge_index=edge_index,noncausal_edge_weight=edge_weight_c,batch=batch)
    (causal_edge_index,causal_edge_weight,topnode),(conf_edge_index,conf_edge_weight,notopnode)=split_graph_bybatch(edge_weight_o,xo_node_att,xc_node_att,edge_index,batch)
    #causalnodes = torch.unique(causal_edge_index).to(device)
   # noncausalnodes = torch.unique(conf_edge_index).to(device)
    #only_causl=torch.zeros(xo.shape).to(device)
    #only_causl[topnode]=xo[topnode]
    only_non=torch.zeros(xc.shape).to(device)
    only_non[notopnode]=xc[notopnode]
    #causal,edge_index,causal_edge_weight,batch
    sub,edge_i,_,edge_w=k_hop_subgraph(topnode,2,causal_edge_index,causal_edge_weight,relabel_nodes=True)
    o_logs2=predictco_model.objects_readout_layer(causal=xo[sub],edge_index=edge_i,causal_edge_weight=edge_w,batch=batch[sub])
    c_logs2=predictco_model.context_readout_layer(noncausal=only_non,edge_index=conf_edge_index,noncausal_edge_weight=conf_edge_weight,batch=batch)

    uniform_target = torch.ones_like(c_logs, dtype=torch.float).to(device) /number_of_class

    #causal part uncertainty
    #target,causal,node,edge_indexn,causal_edge_weightn,batch,c_or_uc='c'
    final_conf_uncer=predictco_model.uncertainty_L_hop(one_hot_target,xo,topnode,causal_edge_index,causal_edge_weight,batch)
    #non-causal part uncertainty
    #target,unfiom_target,noncausal,node,edge_indexn,noncausal_edge_weightn,batch,c_or_uc='c')
    final_conf_uncer_un=predictco_model.non_uncertainty_L_hop(one_hot_target,xc,notopnode,conf_edge_index,conf_edge_weight,batch,c_or_uc='c')

    c_loss = F.kl_div(c_logs, uniform_target, reduction='batchmean')
    #print(f'causal loss{c_loss}')
    o_loss = F.nll_loss(o_logs, one_hot_target)

    c_loss2 = F.kl_div(c_logs2, uniform_target, reduction='batchmean')
    #print(f'causal loss{c_loss}')
    o_loss2 = F.nll_loss(o_logs2, one_hot_target)
    #print(f'noncausal loss{o_loss}')
    co_loss = F.nll_loss(co_logs, one_hot_target)
    #print(f'com causal loss{co_loss}')
    final_conf_uncer.requires_grad_()
    final_conf_uncer_un.requires_grad_()
   # print(c_loss)
    #print(o_loss)
    #print(co_loss)
    #print('-----')
    #print(final_conf_uncer)
    #print(final_conf_uncer_un)
    loss = c * c_loss + o * o_loss + co * co_loss+final_conf_uncer+final_conf_uncer_un+c * c_loss2 +1* o_loss2#+1.5*final_conf_uncer+co*final_conf_uncer_un
    #print(loss)

    pred_o = o_logs.max(1)[1]
    correct_o += pred_o.eq(data.y.view(-1)).sum().item()
    #scaler.scale(loss).backward()
    loss.backward()
    total_loss += loss.item() #* num_graphs(data)
    total_loss_c += c_loss.item() #* num_graphs(data)
    total_loss_o += o_loss.item() #* num_graphs(data)
    total_loss_co += co_loss.item()# * num_graphs(data)
    total_loss_c2 += c_loss2.item() #* num_graphs(data)
    total_loss_o2 += o_loss2.item() #* num_graphs(data)
    o_confin += final_conf_uncer.item()# * num_graphs(data)
    c_unfin+=final_conf_uncer_un.item()
    model_optimizer.step()
    #scaler.step(model_optimizer)
    #scaler.update()
    loop.set_description(f"Epoch [{epoch}/{Epo}] ")
    loop.set_postfix(loss = loss.item())


  #num = len(train_loader.dataset)
  total_loss = total_loss / nb
  #total_loss = total_loss.requires_grad_()
  total_loss_c = total_loss_c / nb
  total_loss_o=total_loss_o/nb
  total_loss_co=total_loss_co/nb
  total_loss_c2=total_loss_c2/nb
  total_loss_o2=total_loss_o2/nb
  o_confin=o_confin/nb
  c_unfin=c_unfin/nb

  print(f'number of {epoch} with total loss:{total_loss}')
  end = time.time()
  print(end - start)
  loss_value.append(total_loss)
  orio.append(total_loss_o)
  orin.append(total_loss_c)
  ocrin.append(total_loss_co)
  only_c.append(total_loss_o2)
  only_n.append(total_loss_c2)
  cer.append(o_confin)
  ucer.append(c_unfin)
  total_loss_o = total_loss_o / nb
  total_loss_co = total_loss_co / nb
  correct_o = correct_o / nb
  lr_scheduler.step()
  # model_optimizer.zero_grad()
  # with torch.autograd.set_detect_anomaly(True):
  #  total_loss.backward()
  #model_optimizer.step()

  with torch.no_grad():
    correct=0
    correct_c=0
    correct_o=0
    correct_o_pred=0
    print(f"------tvalation---------{epoch}")
    causal_model.eval()
    predictco_model.eval()
    for data in val_loader:
      data = data.to(device)
      (xo,xo_node_att,edge_weight_o),(xc,xc_node_att,edge_weight_c),edge_index,batch=causal_model(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      #co_logs = predictredom_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      #causal attention graph prediction
      o_logs,c_logs,co_logs=predictco_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      (causal_edge_index,causal_edge_weight,topnode),(conf_edge_index,conf_edge_weight,notopnode)=split_graph_bybatch(edge_weight_o,xo_node_att,xc_node_att,edge_index,batch,ratio=0.5)
      sub,edge_i,_,edge_w=k_hop_subgraph(topnode,2,causal_edge_index,causal_edge_weight,relabel_nodes=True)
      o_logs2=predictco_model.objects_readout_layer(causal=xo[sub],edge_index=edge_i,causal_edge_weight=edge_w,batch=batch[sub])
      #causal,edge_index,causal_edge_weight,batch
     # o_logs2=predictco_model.objects_readout_layer(causal=only_causl,edge_index=causal_edge_index,causal_edge_weight=causal_edge_weight,batch=batch)
      #c_logs=predictno_model(noncausal=xc,edge_index=edge_index,noncausal_edge_weight=edge_weight_c,batch=batch)

      pred = co_logs.max(1)[1]
      pred_c = c_logs.max(1)[1]
      pred_o = o_logs.max(1)[1]
      pre_ooo=o_logs2.max(1)[1]
      correct += pred.eq(data.y.view(-1)).sum().item()
      correct_c += pred_c.eq(data.y.view(-1)).sum().item()
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
      correct_o_pred+=pre_ooo.eq(data.y.view(-1)).sum().item()
    acc_co = correct / len(val_loader.dataset)
    acc_c = correct_c / len(val_loader.dataset)
    acc_o = correct_o / len(val_loader.dataset)
    acc_only=correct_o_pred/len(val_loader.dataset)
    print(f"causal val accuracy:{acc_co}")
    print(f"causal val only causal accuracy:{acc_o}")
    print(f"causal val only causal accuracy:{acc_only}")
    loss_value_valation.append(acc_only)
    dictionary={"number of epoch":epoch,
                "training loss list":loss_value,
                "valation accuracy list":loss_value_valation}

    # Serializing json
    json_object = json.dumps(dictionary, indent=11)
    #/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose
    # Writing to sample.json
    with open("/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose/mygsst_new.json", "w") as outfile:
        outfile.write(json_object)

    #torch.save(causal_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430cau_my6000.pt')
    #torch.save(predictco_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430caupred_my6000.pt')
    #torch.save(predictno_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430noncaupred_my6000.pt')
    #torch.save(model_heter.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430heter6000.pt')
    torch.save({
            'causal_model.state_dic': causal_model.state_dict(),
            'predictco_model_state_dict()': predictco_model.state_dict(),
            'opt':model_optimizer.state_dict()
            }, '/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose/mygsst_new.pt')
    if(acc_only>83):
        break


  #model_optimizer.zero_grad()
  #total_loss.backward()
  #model_optimizer.step()




  0%|          | 0/222 [00:00<?, ?it/s]

-----training-------0


Epoch [0/100] : 100%|██████████| 222/222 [06:38<00:00,  1.80s/it, loss=1.55]


number of 0 with total loss:1.7651043024149027
398.76875615119934
------tvalation---------0
causal val accuracy:0.7938236489232019
causal val only causal accuracy:0.7259650548557497
causal val only causal accuracy:0.7156440471353108


  0%|          | 0/222 [00:00<?, ?it/s]

...

number of 12 with total loss:0.8517119313951012
419.362610578537
------tvalation---------12
causal val accuracy:0.8208045509955303
causal val only causal accuracy:0.8185290532303942
causal val only causal accuracy:0.8210483543275091


  0%|          | 0/222 [00:00<?, ?it/s]

-----training-------13


Epoch [13/100] :   1%|▏         | 3/222 [00:06<07:19,  2.01s/it, loss=0.944]


KeyboardInterrupt: ignored

In [16]:
(-10)%3

2

In [11]:
#/content/drive/MyDrive/running_mnistme/minemodel_prunepart080808222wo_com
import torch.nn.utils.prune as prune
parameters_to_prune = (
    (predictco_model.object_mlp[1], 'weight'),
    (predictco_model.object_mlp[4], 'weight'),
    (causal_model.edge_att_mlp,'weight'),
    (causal_model.node_att_mlp,'weight'),
    (predictco_model.context_mlp[1], 'weight'),
    (predictco_model.context_mlp[4], 'weight'),
    (predictco_model.random_readout_mlp[1], 'weight'),
    (predictco_model.random_readout_mlp[4], 'weight'),

)
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.3,
)

torch.save({
          'edge_att_mlp.weight_mask': causal_model.state_dict()['edge_att_mlp.weight_mask'],
            'edge_att_mlp.weight_orig': causal_model.state_dict()['edge_att_mlp.weight_orig'],
          'node_att_mlp.weight_mask': causal_model.state_dict()['node_att_mlp.weight_mask'],
            'node_att_mlp.weight_orig': causal_model.state_dict()['node_att_mlp.weight_orig'],
          # 'object_mlp.1.weight_mask': predictco_model.state_dict()['object_mlp.1.weight_mask'],
          'object_mlp.4.weight_mask': predictco_model.state_dict()['object_mlp.4.weight_mask'],
            'object_mlp.4.weight_orig': predictco_model.state_dict()['object_mlp.4.weight_orig'],
          # 'context_mlp.1.weight_mask': predictno_model.state_dict()['context_mlp.1.weight_mask'],
          'context_mlp.4.weight_mask': predictco_model.state_dict()['context_mlp.4.weight_mask'],
            'context_mlp.4.weight_orig': predictco_model.state_dict()['context_mlp.4.weight_orig'],
          #'random_readout_mlp.1.weight_mask': predictno_model.state_dict()['random_readout_mlp.1.weight_mask'],
            'random_readout_mlp.4.weight_mask': predictco_model.state_dict()['random_readout_mlp.4.weight_mask'],
            'random_readout_mlp.4.weight_orig': predictco_model.state_dict()['random_readout_mlp.4.weight_orig']
          }, '/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose/Gsstprune_mas.pt')


for i in parameters_to_prune:
  prune.remove(i[0], 'weight')
loss_value2=[]
loss_value_valation2=[]


In [12]:
#use the node attention
loss_value=[]
orio=[]
orin=[]
ocrin=[]
only_c=[]
only_n=[]
cer=[]
ucer=[]
hetr=[]

loss_value_valation=[]

loss_value2=[]
loss_value_valation2=[]
c=0.5
o=1
co=0.5
import time
import json
import gc
from tqdm import tqdm
Epo=100
#scaler = torch.cuda.amp.GradScaler()

for epoch in range(Epo):
  #model.train()
  total_loss = 0
  total_loss_c = 0
  total_loss_o = 0
  total_loss_co = 0
  total_loss_c2 = 0 #c_loss2.item() #* num_graphs(data)
  total_loss_o2 = 0 #o_loss2.item() #* num_graphs(data)
  o_confin =0 #final_conf_uncer.item()# * num_graphs(data)
  c_unfin =0 #final_conf_uncer_un.item()
  heter_all=0 #heter_loss.item()
  correct_o = 0
  nb=0
  start = time.time()
  loop = tqdm(enumerate(train_loader), total=len(train_loader))
  causal_model.train()
  predictco_model.train()
  print(f"-----training-------{epoch}")
  for it, data in loop:

    nb+=1
    model_optimizer.zero_grad()
    data = data.to(device)

    one_hot_target = data.y.view(-1)
    #causal and non-causal attention graph generate
    # with torch.autocast(device_type='cuda', dtype=torch.float16):
    (xo,xo_node_att,edge_weight_o),(xc,xc_node_att,edge_weight_c),edge_index,batch=causal_model(data)
    #non-causal attention graph prediction
    #xo_logis,xc_logis,xco_logis
    #causal attention graph prediction


    o_logs,c_logs,co_logs=predictco_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
    #c_logs=predictno_model(noncausal=xc,edge_index=edge_index,noncausal_edge_weight=edge_weight_c,batch=batch)
    (causal_edge_index,causal_edge_weight,topnode),(conf_edge_index,conf_edge_weight,notopnode)=split_graph_bybatch(edge_weight_o,xo_node_att,xc_node_att,edge_index,batch)
    #causalnodes = torch.unique(causal_edge_index).to(device)
   # noncausalnodes = torch.unique(conf_edge_index).to(device)
    #only_causl=torch.zeros(xo.shape).to(device)
    #only_causl[topnode]=xo[topnode]
    only_non=torch.zeros(xc.shape).to(device)
    only_non[notopnode]=xc[notopnode]
    #causal,edge_index,causal_edge_weight,batch
    sub,edge_i,_,edge_w=k_hop_subgraph(topnode,2,causal_edge_index,causal_edge_weight,relabel_nodes=True)
    o_logs2=predictco_model.objects_readout_layer(causal=xo[sub],edge_index=edge_i,causal_edge_weight=edge_w,batch=batch[sub])
    c_logs2=predictco_model.context_readout_layer(noncausal=only_non,edge_index=conf_edge_index,noncausal_edge_weight=conf_edge_weight,batch=batch)

    uniform_target = torch.ones_like(c_logs, dtype=torch.float).to(device) /number_of_class

    #causal part uncertainty
    #target,causal,node,edge_indexn,causal_edge_weightn,batch,c_or_uc='c'
    final_conf_uncer=predictco_model.uncertainty_L_hop(one_hot_target,xo,topnode,causal_edge_index,causal_edge_weight,batch,c_or_uc='UC')
    #non-causal part uncertainty
    #target,unfiom_target,noncausal,node,edge_indexn,noncausal_edge_weightn,batch,c_or_uc='c')
    final_conf_uncer_un=predictco_model.non_uncertainty_L_hop(one_hot_target,xc,notopnode,conf_edge_index,conf_edge_weight,batch,c_or_uc='UC')

    c_loss = F.kl_div(c_logs, uniform_target, reduction='batchmean')
    #print(f'causal loss{c_loss}')
    o_loss = F.nll_loss(o_logs, one_hot_target)

    c_loss2 = F.kl_div(c_logs2, uniform_target, reduction='batchmean')
    #print(f'causal loss{c_loss}')
    o_loss2 = F.nll_loss(o_logs2, one_hot_target)
    #print(f'noncausal loss{o_loss}')
    co_loss = F.nll_loss(co_logs, one_hot_target)
    #print(f'com causal loss{co_loss}')
    final_conf_uncer.requires_grad_()
    final_conf_uncer_un.requires_grad_()
   # print(c_loss)
   # print(o_loss)
   # print(co_loss)
    #print('-----')
    #print(final_conf_uncer)
    #print(het_loss)
   # print(final_conf_uncer_un)
    loss = c * c_loss + o * o_loss + co * co_loss+final_conf_uncer+final_conf_uncer_un+c * c_loss2 +1* o_loss2#+1.5*final_conf_uncer+co*final_conf_uncer_un
    #print(loss)

    pred_o = o_logs.max(1)[1]
    correct_o += pred_o.eq(data.y.view(-1)).sum().item()
    #scaler.scale(loss).backward()
    loss.backward()
    total_loss += loss.item() #* num_graphs(data)
    total_loss_c += c_loss.item() #* num_graphs(data)
    total_loss_o += o_loss.item() #* num_graphs(data)
    total_loss_co += co_loss.item()# * num_graphs(data)
    total_loss_c2 += c_loss2.item() #* num_graphs(data)
    total_loss_o2 += o_loss2.item() #* num_graphs(data)
    o_confin += final_conf_uncer.item()# * num_graphs(data)
    c_unfin+=final_conf_uncer_un.item()
    model_optimizer.step()
    #scaler.step(model_optimizer)
    #scaler.update()
    loop.set_description(f"Epoch [{epoch}/{Epo}] ")
    loop.set_postfix(loss = loss.item())


  #num = len(train_loader.dataset)
  total_loss = total_loss / nb
  #total_loss = total_loss.requires_grad_()
  total_loss_c = total_loss_c / nb
  total_loss_o=total_loss_o/nb
  total_loss_co=total_loss_co/nb
  total_loss_c2=total_loss_c2/nb
  total_loss_o2=total_loss_o2/nb
  o_confin=o_confin/nb
  c_unfin=c_unfin/nb

  print(f'number of {epoch} with total loss:{total_loss}')
  end = time.time()
  print(end - start)
  loss_value.append(total_loss)
  orio.append(total_loss_o)
  orin.append(total_loss_c)
  ocrin.append(total_loss_co)
  only_c.append(total_loss_o2)
  only_n.append(total_loss_c2)
  cer.append(o_confin)
  ucer.append(c_unfin)
  total_loss_o = total_loss_o / nb
  total_loss_co = total_loss_co / nb
  correct_o = correct_o / nb
  lr_scheduler.step()
  # model_optimizer.zero_grad()
  # with torch.autograd.set_detect_anomaly(True):
  #  total_loss.backward()
  #model_optimizer.step()

  with torch.no_grad():
    correct=0
    correct_c=0
    correct_o=0
    correct_o_pred=0
    print(f"------tvalation---------{epoch}")
    causal_model.eval()
    predictco_model.eval()
    for data in val_loader:
      data = data.to(device)
      (xo,xo_node_att,edge_weight_o),(xc,xc_node_att,edge_weight_c),edge_index,batch=causal_model(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      #co_logs = predictredom_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      #causal attention graph prediction
      o_logs,c_logs,co_logs=predictco_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      (causal_edge_index,causal_edge_weight,topnode),(conf_edge_index,conf_edge_weight,notopnode)=split_graph_bybatch(edge_weight_o,xo_node_att,xc_node_att,edge_index,batch,ratio=0.5)
      sub,edge_i,_,edge_w=k_hop_subgraph(topnode,2,causal_edge_index,causal_edge_weight,relabel_nodes=True)
      o_logs2=predictco_model.objects_readout_layer(causal=xo[sub],edge_index=edge_i,causal_edge_weight=edge_w,batch=batch[sub])
      #causal,edge_index,causal_edge_weight,batch
     # o_logs2=predictco_model.objects_readout_layer(causal=only_causl,edge_index=causal_edge_index,causal_edge_weight=causal_edge_weight,batch=batch)
      #c_logs=predictno_model(noncausal=xc,edge_index=edge_index,noncausal_edge_weight=edge_weight_c,batch=batch)

      pred = co_logs.max(1)[1]
      pred_c = c_logs.max(1)[1]
      pred_o = o_logs.max(1)[1]
      pre_ooo=o_logs2.max(1)[1]
      correct += pred.eq(data.y.view(-1)).sum().item()
      correct_c += pred_c.eq(data.y.view(-1)).sum().item()
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
      correct_o_pred+=pre_ooo.eq(data.y.view(-1)).sum().item()
    acc_co = correct / len(val_loader.dataset)
    acc_c = correct_c / len(val_loader.dataset)
    acc_o = correct_o / len(val_loader.dataset)
    acc_only=correct_o_pred/len(val_loader.dataset)
    print(f"causal val accuracy:{acc_co}")
    print(f"causal val only causal accuracy:{acc_o}")
    print(f"causal val only causal accuracy:{acc_only}")
    loss_value_valation.append(acc_only)
    dictionary={"number of epoch":epoch,
                "training loss list":loss_value,
                "valation accuracy list":loss_value_valation}

    # Serializing json
    json_object = json.dumps(dictionary, indent=11)
    #/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose
    # Writing to sample.json
    with open("/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose/mygsst_new.json", "w") as outfile:
        outfile.write(json_object)

    #torch.save(causal_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430cau_my6000.pt')
    #torch.save(predictco_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430caupred_my6000.pt')
    #torch.save(predictno_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430noncaupred_my6000.pt')
    #torch.save(model_heter.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430heter6000.pt')
    torch.save({
            'causal_model.state_dic': causal_model.state_dict(),
            'predictco_model_state_dict()': predictco_model.state_dict(),
            'opt':model_optimizer.state_dict()
            }, '/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose/after_mygsst_new.pt')
    if(acc_only>83):
        break


  #model_optimizer.zero_grad()
  #total_loss.backward()
  #model_optimizer.step()

  0%|          | 0/222 [00:00<?, ?it/s]

-----training-------0


Epoch [0/100] : 100%|██████████| 222/222 [03:50<00:00,  1.04s/it, loss=0.535]


number of 0 with total loss:0.6439479819020709
230.39638257026672
------tvalation---------0
causal val accuracy:0.7931735067045916
causal val only causal accuracy:0.7911418122714344
causal val only causal accuracy:0.802519301097115


  0%|          | 0/222 [00:00<?, ?it/s]

...

number of 98 with total loss:0.4243584875319455
235.4114215373993
------tvalation---------98
causal val accuracy:0.8014628199918732
causal val only causal accuracy:0.7900040633888663
causal val only causal accuracy:0.7891913856156034


  0%|          | 0/222 [00:00<?, ?it/s]

-----training-------99


Epoch [99/100] : 100%|██████████| 222/222 [03:56<00:00,  1.07s/it, loss=0.561]


number of 99 with total loss:0.4170858464665241
236.91299033164978
------tvalation---------99
causal val accuracy:0.8070702966273873
causal val only causal accuracy:0.7997561966680211
causal val only causal accuracy:0.8008939455505892


In [12]:
checkpoint1 = torch.load('/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose/Gsstprune_mas.pt')
def setpara(input_para,mask,set_para):
  w=mask.nonzero()
  for i in w:
    input_para[i[0],i[1]]=set_para[i[0],i[1]]

#causal_model.state_dict()['edge_att_mlp.weight']
setpara(causal_model.state_dict()['edge_att_mlp.weight'],checkpoint1['edge_att_mlp.weight_mask'],checkpoint1['edge_att_mlp.weight_orig'])
setpara(causal_model.state_dict()['node_att_mlp.weight'],checkpoint1['node_att_mlp.weight_mask'],checkpoint1['node_att_mlp.weight_orig'])
setpara(predictco_model.state_dict()['object_mlp.4.weight'],checkpoint1['object_mlp.4.weight_mask'],checkpoint1['object_mlp.4.weight_orig'])
setpara(predictco_model.state_dict()['context_mlp.4.weight'],checkpoint1['context_mlp.4.weight_mask'],checkpoint1['context_mlp.4.weight_orig'])
setpara(predictco_model.state_dict()['random_readout_mlp.4.weight'],checkpoint1['random_readout_mlp.4.weight_mask'],checkpoint1['random_readout_mlp.4.weight_orig'])

torch.save({
        'causal_model.state_dic': causal_model.state_dict(),
        'predictco_model_state_dict()': predictco_model.state_dict(),
        }, '/content/drive/MyDrive/running_RQ2/running_Graph_sst2/running_mypropose/finalminemodel.pt')


In [ ]:
'''torch.save(causal_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430cau_my6000.pt')
torch.save(predictco_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430caupred_my6000.pt')
torch.save(predictno_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430noncaupred_my6000.pt')
torch.save(model_heter.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430heter6000.pt')'''

In [ ]:
'''import torch.nn.utils.prune as prune
parameters_to_prune = (
    (predictco_model.object_mlp[1], 'weight'),
    (predictco_model.object_mlp[3], 'weight'),
    (predictco_model.objects_convs.lin, 'weight'),
    (causal_model.edge_att_mlp,'weight'),
    (causal_model.node_att_mlp,'weight'),
    (predictno_model.context_mlp[1], 'weight'),
    (predictno_model.context_mlp[3], 'weight'),
    (predictno_model.objects_convs.lin, 'weight'),
    (predictno_model.random_readout_mlp[1], 'weight'),
    (predictno_model.random_readout_mlp[3], 'weight'),
    (predictno_model.context_convs.lin, 'weight'),

)
parameters_to_prune'''

In [ ]:
'''prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)
for i in parameters_to_prune:
  prune.remove(i[0], 'weight')'''

In [ ]:
'''loss_value_train=[]
loss_value_valation=[]
c=0.5
o=1
co=0.5
import time
from tqdm import tqdm
scaler = torch.cuda.amp.GradScaler()
def num_graphs(data):
  if data.batch is not None:
      return data.num_graphs
  else:
      return data.x.size(0)
for epoch in range(300):
  #model.train()
  total_loss = 0
  total_loss_c = 0
  total_loss_o = 0
  total_loss_co = 0
  correct_o = 0
  nb=0
  start = time.time()
  loop = tqdm(enumerate(train_loader), total=len(train_loader))
  print(f"-----training-------{epoch}")
  for it, data in loop:
      nb+=1
      model_optimizer.zero_grad()
      data = data.to(device)

      one_hot_target = data.y.view(-1)
      #causal and non-causal attention graph generate
     # with torch.autocast(device_type='cuda', dtype=torch.float16):
      (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model(data)
      #non-causal attention graph prediction
      c_logs,co_logs = predictno_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      #causal attention graph prediction
      o_logs=predictco_model(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
      #ground trouth heterogeneous edge mask generation
      ground_heter_o=model_heter.get_label(edge_index,xo,batch).to(device)
      het_c=model_heter(graph=xc,edge_index=edge_index,batch=batch).to(device)
      #ground_heter_o=model_heter.get_label(edge_index,xo,batch)
      het_o=model_heter(graph=xo,edge_index=edge_index,batch=batch).to(device)
      fin_het=(het_c+het_o)/2
      edge_hetero_mask=m(fin_het).reshape(-1).round().to(device)
      #if(epoch>50):
      t1=torch.masked_select(edge_index[0],edge_hetero_mask==0).to(device)
      t2=torch.masked_select(edge_index[1],edge_hetero_mask==0).to(device)
      homo_index=torch.stack([t1,t2]).to(device)
      edweig_non=torch.masked_select(edge_weight_c,edge_hetero_mask==0).to(device)
      edweig_coa=torch.masked_select(edge_weight_o,edge_hetero_mask==0).to(device)
      #else:
      # t1=torch.masked_select(edge_index[0],ground_heter_o==0).to(device)
      # t2=torch.masked_select(edge_index[1],ground_heter_o==0).to(device)
      # homo_index=torch.stack([t1,t2]).to(device)
      # edweig_non=torch.masked_select(edge_weight_c,ground_heter_o==0).to(device)
      # edweig_coa=torch.masked_select(edge_weight_o,ground_heter_o==0).to(device)


      uniform_target = torch.ones_like(c_logs, dtype=torch.float).to(device) /number_of_class
      #causal part uncertainty
      final_conf_uncer=predictco_model.uncertainty_L_hop(data,xo,edge_index,edge_weight_o,batch,homo_index,edweig_coa)
      #non-causal part uncertainty
      final_conf_uncer_un=predictno_model.non_uncertainty_L_hop(uniform_target,xc,edge_index,edge_weight_c,batch,homo_index,edweig_non)

      het_loss=loss_he_ho(fin_het.to(device),ground_heter_o.unsqueeze(1).to(device))
      c_loss = F.kl_div(c_logs, uniform_target, reduction='batchmean')
      #print(f'causal loss{c_loss}')
      o_loss = F.nll_loss(o_logs, one_hot_target)
      #print(f'noncausal loss{o_loss}')
      co_loss = F.nll_loss(co_logs, one_hot_target)
      #print(f'com causal loss{co_loss}')
      #print(c_loss)
      #print(o_loss)
      #print(co_loss)
    # print(final_conf_uncer)
    # print(het_loss)
      #print(final_conf_uncer_un)
      loss = c * c_loss + o * o_loss + co * co_loss+het_loss+1.5*final_conf_uncer+co*final_conf_uncer_un
      #print(loss)
      pred_o = o_logs.max(1)[1]
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
      #scaler.scale(loss).backward()
      loss.backward()
      total_loss += loss#.item() #* num_graphs(data)
      total_loss_c += c_loss#.item() #* num_graphs(data)
      total_loss_o += o_loss#.item() #* num_graphs(data)
      total_loss_co += co_loss#.item()# * num_graphs(data)
      model_optimizer.step()
      #scaler.step(model_optimizer)
      #scaler.update()
      loop.set_description(f"Epoch [{epoch+1}/{300}] ")
      loop.set_postfix(loss = loss.item())


  #num = len(train_loader.dataset)
  total_loss = total_loss / nb
  #total_loss = total_loss.requires_grad_()
  total_loss_c = total_loss_c / nb
  print(f'number of {epoch} with total loss:{total_loss}')
  end = time.time()
  print(end - start)
  loss_value_train.append(total_loss)
  total_loss_o = total_loss_o / nb
  total_loss_co = total_loss_co / nb
  correct_o = correct_o / nb
 # model_optimizer.zero_grad()
 # with torch.autograd.set_detect_anomaly(True):
  #  total_loss.backward()
  #model_optimizer.step()
  with torch.no_grad():
    correct=0
    correct_c=0
    correct_o=0
    correct_o_pred=0
    print(f"------valation---------{epoch}")
    causal_model.eval()
    predictno_model.eval()
    predictco_model.eval()
    for data in val_loader:
      data = data.to(device)
      (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      c_logs,co_logs = predictno_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      o_logs=predictco_model(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
      node_of_graph=xo.shape[0]
      g,node,index,weight=split_graph(graph_x=xo,node_of_graph=node_of_graph,edge_index=edge_index,edge_weight=edge_weight_o,type_of_graph=True)
      o_logs_pred=predictco_model(causal=g,edge_index=index,causal_edge_weight=weight,batch=batch)

      pred = co_logs.max(1)[1]
      pred_c = c_logs.max(1)[1]
      pred_o = o_logs.max(1)[1]
      pred_o_p = o_logs_pred.max(1)[1]

      correct += pred.eq(data.y.view(-1)).sum().item()
      correct_c += pred_c.eq(data.y.view(-1)).sum().item()
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
      correct_o_pred+= pred_o_p.eq(data.y.view(-1)).sum().item()
    acc_co = correct / len(val_loader.dataset)
    acc_c = correct_c / len(val_loader.dataset)
    acc_o = correct_o / len(val_loader.dataset)
    acc_o_pred = correct_o_pred / len(val_loader.dataset)
    print(f"causal val accuracy:{acc_o}")
    print(f"causal val only causal accuracy:{acc_o_pred}")
    loss_value_valation.append(acc_o)
    if(epoch>100):
      check=abs(acc_o-loss_value_valation[len(loss_value_valation)-10])/10
      if(check<=0.00001):
        break


  #model_optimizer.zero_grad()
  #total_loss.backward()'''
  #model_optimizer.step()


In [ ]:
loss_value_train=[]
loss_value_valation=[]
c=0.5
o=1
co=0.5
import time
import json
import gc
from tqdm import tqdm
Epo=500
#scaler = torch.cuda.amp.GradScaler()
def num_graphs(data):
  if data.batch is not None:
      return data.num_graphs
  else:
      return data.x.size(0)
for epoch in range(67,Epo):
  #model.train()
  total_loss = 0
  total_loss_c = 0
  total_loss_o = 0
  total_loss_co = 0
  correct_o = 0
  nb=0
  start = time.time()
  loop = tqdm(enumerate(train_loader), total=len(train_loader))
  print(f"-----training-------{epoch}")
  for it, data in loop:
    nb+=1
    model_optimizer.zero_grad()
    data = data.to(device)

    one_hot_target = data.y.view(-1)
    #causal and non-causal attention graph generate
    # with torch.autocast(device_type='cuda', dtype=torch.float16):
    (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model(data)
    #non-causal attention graph prediction
    c_logs,co_logs = predictno_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
    #causal attention graph prediction
    o_logs=predictco_model(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
    #ground trouth heterogeneous edge mask generation
   # ground_heter_o=model_heter.get_label(edge_index,xo,batch).to(device)
    het_c=model_heter(graph=xc,edge_index=edge_index,batch=batch).to(device)
    ground_heter_o=model_heter.get_label(edge_index,xo,batch)
    het_o=model_heter(graph=xo,edge_index=edge_index,batch=batch).to(device)
    fin_het=(het_c+het_o)/2
    edge_hetero_mask=m(fin_het).reshape(-1).round().to(device)
    #if(epoch>50):
    t1=torch.masked_select(edge_index[0],edge_hetero_mask==1).to(device)
    t2=torch.masked_select(edge_index[1],edge_hetero_mask==1).to(device)
    heter_index=torch.stack([t1,t2]).to(device)
    edweig_non_het=torch.masked_select(edge_weight_c,edge_hetero_mask==1).to(device)
    edweig_coa_het=torch.masked_select(edge_weight_o,edge_hetero_mask==1).to(device)
    #else:
    # t1=torch.masked_select(edge_index[0],ground_heter_o==0).to(device)
    # t2=torch.masked_select(edge_index[1],ground_heter_o==0).to(device)
    # homo_index=torch.stack([t1,t2]).to(device)
    # edweig_non=torch.masked_select(edge_weight_c,ground_heter_o==0).to(device)
    # edweig_coa=torch.masked_select(edge_weight_o,ground_heter_o==0).to(device)
    node_of_graph=xc.shape[0]
    only_non_part,non_node,non_index,non_weight=split_graph(graph_x=xc,node_of_graph=node_of_graph,edge_index=edge_index,edge_weight=edge_weight_c,type_of_graph=False)
    only_cau_part,cau_node,cau_index,cau_weight=split_graph(graph_x=xo,node_of_graph=node_of_graph,edge_index=edge_index,edge_weight=edge_weight_o,type_of_graph=True)

    cau_index,cau_weight=mask_heter2(heter_index,edweig_coa_het,cau_index,cau_weight)
    non_index,non_weight=mask_heter2(heter_index,edweig_non_het,non_index,non_weight)


    o_logs2=predictco_model.objects_readout_layer(causal=only_cau_part,edge_index=cau_index,causal_edge_weight=cau_weight,batch=batch)
    c_logs2=predictno_model.context_readout_layer(noncausal=only_non_part,edge_index=non_index,noncausal_edge_weight=non_weight,batch=batch)

    uniform_target = torch.ones_like(c_logs, dtype=torch.float).to(device) /number_of_class
    #causal part uncertainty
    #final_conf_uncer=predictco_model.uncertainty_L_hop(data,xo,edge_index,edge_weight_o,batch,homo_index,edweig_coa)
    #non-causal part uncertainty
    #final_conf_uncer_un=predictno_model.non_uncertainty_L_hop(uniform_target,xc,edge_index,edge_weight_c,batch,homo_index,edweig_non)

    het_loss=loss_he_ho(fin_het.to(device),ground_heter_o.unsqueeze(1).to(device))
    c_loss = F.kl_div(c_logs, uniform_target, reduction='batchmean')
    #print(f'causal loss{c_loss}')
    o_loss = F.nll_loss(o_logs, one_hot_target)

    c_loss2 = F.kl_div(c_logs2, uniform_target, reduction='batchmean')
    #print(f'causal loss{c_loss}')
    o_loss2 = F.nll_loss(o_logs2, one_hot_target)
    #print(f'noncausal loss{o_loss}')
    co_loss = F.nll_loss(co_logs, one_hot_target)
    #print(f'com causal loss{co_loss}')
    #print(c_loss)
    #print(o_loss)
    #print(co_loss)
  # print(final_conf_uncer)
  # print(het_loss)
    #print(final_conf_uncer_un)
    loss = c * c_loss + o * o_loss + co * co_loss+c * c_loss2 + o * o_loss2+het_loss#+1.5*final_conf_uncer+co*final_conf_uncer_un
    #print(loss)
    pred_o = o_logs.max(1)[1]
    correct_o += pred_o.eq(data.y.view(-1)).sum().item()
    #scaler.scale(loss).backward()
    loss.backward()
    total_loss += loss#.item() #* num_graphs(data)
    total_loss_c += c_loss#.item() #* num_graphs(data)
    total_loss_o += o_loss#.item() #* num_graphs(data)
    total_loss_co += co_loss#.item()# * num_graphs(data)
    model_optimizer.step()
    #scaler.step(model_optimizer)
    #scaler.update()
    loop.set_description(f"Epoch [{epoch+1}/{Epo}] ")
    loop.set_postfix(loss = loss.item())


  #num = len(train_loader.dataset)
  total_loss = total_loss / nb
  #total_loss = total_loss.requires_grad_()
  total_loss_c = total_loss_c / nb
  print(f'number of {epoch} with total loss:{total_loss}')
  end = time.time()
  print(end - start)
  loss_value_train.append(total_loss)
  total_loss_o = total_loss_o / nb
  total_loss_co = total_loss_co / nb
  correct_o = correct_o / nb
 # model_optimizer.zero_grad()
 # with torch.autograd.set_detect_anomaly(True):
  #  total_loss.backward()
  #model_optimizer.step()

  with torch.no_grad():
    correct=0
    correct_c=0
    correct_o=0
    correct_o_pred=0
    print(f"------valation---------{epoch}")
    causal_model.eval()
    predictno_model.eval()
    predictco_model.eval()
    for data in val_loader:
      data = data.to(device)
      (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      c_logs,co_logs = predictno_model(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      o_logs=predictco_model(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
      node_of_graph=xo.shape[0]
      g,node,index,weight=split_graph(graph_x=xo,node_of_graph=node_of_graph,edge_index=edge_index,edge_weight=edge_weight_o,type_of_graph=True)
      o_logs_pred=predictco_model(causal=g,edge_index=index,causal_edge_weight=weight,batch=batch)

      pred = co_logs.max(1)[1]
      pred_c = c_logs.max(1)[1]
      pred_o = o_logs.max(1)[1]
      pred_o_p = o_logs_pred.max(1)[1]

      correct += pred.eq(data.y.view(-1)).sum().item()
      correct_c += pred_c.eq(data.y.view(-1)).sum().item()
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
      correct_o_pred+= pred_o_p.eq(data.y.view(-1)).sum().item()
    acc_co = correct / len(val_loader.dataset)
    acc_c = correct_c / len(val_loader.dataset)
    acc_o = correct_o / len(val_loader.dataset)
    acc_o_pred = correct_o_pred / len(val_loader.dataset)
    print(f"causal val accuracy:{acc_o}")
    print(f"causal val only causal accuracy:{acc_o_pred}")
    loss_value_valation.append(acc_o)
    dictionary={"number of epoch":epoch}

    # Serializing json
    json_object = json.dumps(dictionary, indent=1)

    # Writing to sample.json
    with open("/content/drive/MyDrive/Colab_Notebooks/number_epoch.json", "w") as outfile:
        outfile.write(json_object)

    #torch.save(causal_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430cau_my6000.pt')
    #torch.save(predictco_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430caupred_my6000.pt')
    #torch.save(predictno_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430noncaupred_my6000.pt')
    #torch.save(model_heter.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430heter6000.pt')
    torch.save({
            'causal_model.state_dic': causal_model.state_dict(),
            'predictco_model_state_dict()': predictco_model.state_dict(),
            'predictno_model_state_dict()': predictno_model.state_dict(),
            'model_heter_state_dict()': model_heter.state_dict(),
            'opt':model_optimizer.state_dict()
            }, '/content/drive/MyDrive/Colab_Notebooks/430allmodel.pt')
    if(epoch>100):
      check=abs(acc_o-loss_value_valation[len(loss_value_valation)-10])/10
      if(check<=0.00001):
        break


  #model_optimizer.zero_grad()
  #total_loss.backward()
  #model_optimizer.step()



# Data to be written
#4


In [ ]:
'''torch.save(causal_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/cau_my6000.pt')
torch.save(predictco_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/caupred_my6000.pt')'''

In [ ]:
for i in range(len(loss_value_train)):
  loss_value_train[i]=float(loss_value_train[i])
plt.plot(np.array(loss_value_train))
plt.show()

In [ ]:
for i in range(len(loss_value_valation)):
  loss_value_valation[i]=float(loss_value_valation[i])
plt.plot(np.array(loss_value_valation))
plt.show()